In [1]:
import random, math

In [10]:
def beam_search(env, k=2, max_depth=10, verbose=True):
    beam = [env.initial_state()]
    if verbose: print("Initial state cost:", env.cost(beam[0]))
    
    for depth in range(max_depth):
        candidates = []
        for state in beam:
            candidates.extend(env.neighbours(state))
        if not candidates:
            break
        candidates.sort(key=env.cost)
        beam = candidates[:k]
        
        if verbose:
            print(f"Depth {depth+1}:")
            for i, st in enumerate(beam):
                print(f"  Beam {i+1}: cost={env.cost(st)}")
    
    best = min(beam, key=env.cost)
    return best, env.cost(best)


In [12]:
def simulated_annealing(env, T=10.0, alpha=0.9, iters=100, verbose=True):
    state = env.initial_state()
    best, best_cost = state, env.cost(state)
    
    if verbose: print("Initial cost:", best_cost)
    
    for it in range(iters):
        neigh = env.random_neighbour(state)
        if neigh is None: continue
        delta = env.cost(neigh) - env.cost(state)
        accept = delta < 0 or random.random() < math.exp(-delta/T)
        
        if accept:
            state = neigh
            if env.cost(state) < best_cost:
                best, best_cost = state, env.cost(state)
        
        if verbose and it % 5 == 0:  # print every 5 steps
            print(f"Iter {it:3d}, T={T:.3f}, curr_cost={env.cost(state)}, best_cost={best_cost}")
        
        T *= alpha
    
    return best, best_cost


In [ ]:
import random
from collections import Counter

def genetic_algorithm(env, pop_size=10, gens=20, mutation_rate=0.1, verbose=True):
    population = [env.initial_state() for _ in range(pop_size)]
    best = None
    best_cost = float("inf")
    
    for g in range(gens):
        scored = [(env.cost(ind), ind) for ind in population]
        scored.sort(key=lambda x: x[0])
        
        if scored[0][0] < best_cost:
            best_cost, best = scored[0]
        
        if verbose:
            avg_cost = sum(s for s, _ in scored) / len(scored)
            print(f"Gen {g+1}: best cost={scored[0][0]}, avg cost={avg_cost:.2f}")
        
        # Select top half as parents
        parents = [ind for _, ind in scored[:pop_size // 2]]
        
        children = []
        while len(children) < pop_size:
            p1, p2 = random.sample(parents, 2)
            child = jobshop_crossover(p1, p2)  # custom crossover for jobshop
            
            # Mutation: swap two positions
            if random.random() < mutation_rate:
                i, j = random.sample(range(len(child)), 2)
                child[i], child[j] = child[j], child[i]
            
            children.append(child)
        
        population = children
    
    return best, best_cost


def jobshop_crossover(p1, p2):
    """Order-based crossover preserving job counts"""
    n = len(p1)
    child = []
    # counts of how many times each job already used
    used = Counter()
    # total required counts (from parent1)
    required = Counter(p1)
    
    # iterate positions, choose gene from p1 or p2
    for i in range(n):
        if random.random() < 0.5:
            cand = p1[i]
        else:
            cand = p2[i]
        # if still allowed, take it
        if used[cand] < required[cand]:
            child.append(cand)
            used[cand] += 1
        else:
            # fallback: choose any remaining job with quota left
            for job in required:
                if used[job] < required[job]:
                    child.append(job)
                    used[job] += 1
                    break
    return child
